In [2]:
import os
import sys
parent_dir = os.path.abspath('.')
sys.path.append(parent_dir + '/bilinear_interp_tim')
import argparse
import gc
import json
import math
from collections import defaultdict
import torch as t
from einops import rearrange
from tqdm import tqdm

from activation_utils import SparseAct
from attribution import patching_effect, jvp
from circuit_plotting import plot_circuit, plot_circuit_posaligned
from dictionary_learning import AutoEncoder
from loading_utils import load_examples, load_examples_nopair
from nnsight import LanguageModel
from language import Transformer, Sight
from sae_adopter import DictionarySAE
from bilinear_circuits_v0 import initialize_model_and_dictionaries

In [3]:
device, model, embed, attns, mlps, resids, dictionaries, save_basename, examples, batch_size, num_examples, n_batches, batches = initialize_model_and_dictionaries(
    device='cuda:0',
    model_name="tdooms/fw-nano",
    dict_id='10',  # Note: This was originally an int, but the function expects a string.
    d_model=1024,
    dict_path='tdooms/fw-nano-scope',
    dataset='simple_train',
    num_examples=20,
    example_length=None,
    batch_size=4,
    aggregation='sum',
    nopair=False,
)

Clean and patch inputs of different shapes.
Clean: 3 Patch: 4
Clean and patch inputs of different shapes.
Clean: 4 Patch: 3


In [4]:
t.cuda.empty_cache()

all_submods = [embed] + [submod for layer_submods in zip(mlps, attns, resids) for submod in layer_submods]
clean_inputs = t.cat([e['clean_prefix'] for e in batches[0]], dim=0).to(device)
clean_answer_idxs = t.tensor([e['clean_answer'] for e in batches[0]], dtype=t.long, device=device)

patch_inputs = t.cat([e['patch_prefix'] for e in batches[0]], dim=0).to(device)
patch_answer_idxs = t.tensor([e['patch_answer'] for e in batches[0]], dtype=t.long, device=device)
def metric_fn(model):
    return (
        t.gather(model.lm_head.output[:,-1,:], dim=-1, index=patch_answer_idxs.view(-1, 1)).squeeze(-1) - \
        t.gather(model.lm_head.output[:,-1,:], dim=-1, index=clean_answer_idxs.view(-1, 1)).squeeze(-1)
    ) #We're only looking at the logit difference between two answers which is a very limited subset of the model's behavior aya. 

In [5]:
patch_inputs

tensor([[    1,   415,  4531],
        [    1,   415,  3282],
        [    1,   415,  4649],
        [    1,   415, 13500]], device='cuda:0')

In [6]:
clean_inputs

tensor([[   1,  415, 8066],
        [   1,  415, 1832],
        [   1,  415, 6246],
        [   1,  415, 6676]], device='cuda:0')

In [7]:
%%time
effects, deltas, grads, total_effect = patching_effect(
        clean_inputs,
        patch_inputs,
        model,
        all_submods,
        dictionaries,
        metric_fn,
        metric_kwargs=dict(),
        method='ig' # get better approximations for early layers by using ig
    )


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



Integrated Gradient estimation


Initial trace

torch.Size([4, 3, 1024])
tensor(6.0038, device='cuda:0', grad_fn=<MaxBackward1>)
tensor([[[ 4.2607e-02,  7.5031e-03, -1.7864e-02,  ...,  4.6599e-04,
          -3.8948e-03, -1.8446e-02],
         [ 1.1957e-01,  8.2422e-03,  1.2530e-01,  ..., -1.1062e-01,
           5.9906e-02,  3.3296e-01],
         [ 7.0522e-01,  3.2143e-01,  5.6787e-01,  ..., -4.0202e-01,
           2.5385e+00, -5.4219e-01]],

        [[ 4.2607e-02,  7.5031e-03, -1.7864e-02,  ...,  4.6599e-04,
          -3.8948e-03, -1.8446e-02],
         [ 1.1957e-01,  8.2422e-03,  1.2530e-01,  ..., -1.1062e-01,
           5.9906e-02,  3.3296e-01],
         [ 1.8139e+00, -3.0347e+00, -2.9061e-01,  ...,  1.1858e+00,
          -7.5644e-01,  1.0659e+00]],

        [[ 4.2607e-02,  7.5031e-03, -1.7864e-02,  ...,  4.6599e-04,
          -3.8948e-03, -1.8446e-02],
         [ 1.1957e-01,  8.2422e-03,  1.2530e-01,  ..., -1.1062e-01,
           5.9906e-02,  3.3296e-01],
         [ 1.2066e+00, -7.

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f269d96ae50>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


CPU times: user 6.86 s, sys: 236 ms, total: 7.09 s
Wall time: 7.1 s
